### Задание

Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

### Решение

In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

adults = pd.read_csv('adult_csv.csv')
adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      46033 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capitalgain     48842 non-null  int64 
 11  capitalloss     48842 non-null  int64 
 12  hoursperweek    48842 non-null  int64 
 13  native-country  47985 non-null  object
 14  class           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [3]:
# Постфактум поняла, что можно было использовать LabelEncoder

def workclass(_wcl):
    if _wcl == 'Private':
        return 1
    elif _wcl == 'Self-emp-not-inc':
        return 2
    elif _wcl == 'Local-gov':
        return 3
    elif _wcl == 'State-gov':
        return 4
    elif _wcl == 'Self-emp-inc':
        return 5
    elif _wcl == 'Federal-gov':
        return 6
    elif _wcl == 'Without-pay':
        return 7
    elif _wcl == 'Never-worked':
        return 8
    else:  #unknoun|None
        return 0

def education(_edu):
    if _edu == 'HS-grad':
        return 1
    elif _edu == 'Some-college':
        return 2
    elif _edu == 'Bachelors':
        return 3
    elif _edu == 'Masters':
        return 4
    elif _edu == 'Assoc-voc':
        return 5
    elif _edu == '11th':
        return 6
    elif _edu == 'Assoc-acdm':
        return 7
    elif _edu == '10th':
        return 8
    elif _edu == '7th-8th':
        return 9
    elif _edu == 'Prof-school':
        return 10
    elif _edu == '9th':
        return 11
    elif _edu == '12th':
        return 12
    elif _edu == 'Doctorate':
        return 13
    elif _edu == '5th-6th':
        return 14
    elif _edu == '1st-4th':
        return 15
    elif _edu == 'Preschool':
        return 16
        

def marital_status(_ms):
    if _ms == 'Married-civ-spouse':
        return 1
    elif _ms == 'Never-married':
        return 2
    elif _ms == 'Divorced':
        return 3
    elif _ms == 'Separated':
        return 4
    elif _ms == 'Widowed':
        return 5
    elif _ms == 'Married-spouse-absent':
        return 6
    elif _ms == 'Married-AF-spouse':
        return 7

def race(_race):
    if _race == 'White':
        return 1
    elif _race == 'Black':
        return 2
    elif _race == 'Asian-Pac-Islander':
        return 3
    elif _race == 'Amer-Indian-Eskimo':
        return 4
    elif _race == 'Other':
        return 5

def sex(_s):
    if _s == 'Male':
        return 0
    else:
        return 1

def occupation(_oc):
    if _oc == 'Prof-specialty':
        return 1
    elif _oc == 'Craft-repair':
        return 2
    elif _oc == 'Exec-managerial':
        return 3
    elif _oc == 'Adm-clerical':
        return 4
    elif _oc == 'Sales':
        return 5
    elif _oc == 'Other-service':
        return 6
    elif _oc == 'Machine-op-inspct':
        return 7
    elif _oc == 'Transport-moving':
        return 8
    elif _oc == 'Handlers-cleaners':
        return 9
    elif _oc == 'Farming-fishing':
        return 10
    elif _oc == 'Tech-support':
        return 11
    elif _oc == 'Protective-serv':
        return 12
    elif _oc == 'Priv-house-serv':
        return 13
    elif _oc == 'Armed-Forces':
        return 14
    else:  #unknoun|None
        return 0

def target_class(_c):
    if _c == '<=50K':
        return 1
    else:
        return 2

# переводим текстовые признаки в числовые заполняя пропуски в категориальных атрибутах
adults['sex'] = adults.sex.apply(sex)
adults['race'] = adults.race.apply(race)
adults['workclass'] = adults.workclass.apply(workclass)
adults['education'] = adults.education.apply(education)
adults['marital-status'] = adults['marital-status'].apply(marital_status)
adults['occupation'] = adults.occupation.apply(occupation)
adults['class'] = adults['class'].apply(target_class)


# убираем оставшиеся тексты и отделяем класс как целевую переменную
del adults['relationship']
del adults['native-country']
target = adults['class']
del adults['class']
adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             48842 non-null  int64
 1   workclass       48842 non-null  int64
 2   fnlwgt          48842 non-null  int64
 3   education       48842 non-null  int64
 4   education-num   48842 non-null  int64
 5   marital-status  48842 non-null  int64
 6   occupation      48842 non-null  int64
 7   race            48842 non-null  int64
 8   sex             48842 non-null  int64
 9   capitalgain     48842 non-null  int64
 10  capitalloss     48842 non-null  int64
 11  hoursperweek    48842 non-null  int64
dtypes: int64(12)
memory usage: 4.5 MB


### Logistic Regression

In [4]:
# делим сет на обучающий и тренировочный, строим модель регрессии
X_train, X_test, y_train, y_test = train_test_split(adults, target, test_size=0.35, random_state=42)


model = LogisticRegression()

model.fit(X_train, y_train)

# есть ли разница между использованием score и sklearn.metrics.accuracy_score ? Результаты для здесь равны
print('Точность предсказания на тестовой выборкe:  ', round(model.score(X_test,y_test), 5))
print('Точность предсказания на обучающей выборкe: ', round(model.score(X_train, y_train), 5))

Точность предсказания на тестовой выборкe:   0.75794
Точность предсказания на обучающей выборкe:  0.76221


### SVM

In [ ]:
# Разными вариантами кол-ва ядер/степеней пыталась в цикле. По типу ядра poly здесь точнее sigmoid
svm_model = SVC(kernel = 'poly', degree=7)
svm_model.fit(X_train, y_train)
print(f'Точность предсказания на тестовой выборкe: ',
                      round(svm_model.score(X_test,y_test), 5))
print(f'Точность предсказания на обучающей выборкe: ',
                      round(svm_model.score(X_train, y_train), 5))
# Дефолтные 3 ядра дают тот же результат, что и логистическая регрессия. 
# Увеличение кол-ва степеней увеличивает точность прогноза
# Компьютеру очень тяжело